<a href="https://colab.research.google.com/github/rupaidutta66/Text_Extract_from_PDF/blob/main/Text_Extraction_Hengyang_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y poppler-utils
!pip install pdf2image
!pip install easyocr
!pip install opencv-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 0s (438 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 122352 fi

In [ ]:
# !pip install pytesseract
# !sudo apt install tesseract-ocr

In [ ]:
from pdf2image import convert_from_path
from PIL import ImageDraw
import easyocr
import cv2
#import pytesseract
import numpy as np
import pandas as pd
import re

In [ ]:
reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
image = convert_from_path('/content/Screenshot 2023-03-09 013144.png', dpi=300 )
img = image[1]
bounds = reader.readtext(np.array(img), min_size=0, slope_ths=0.2, ycenter_ths=0.6, width_ths= 0.7, decoder='beamsearch', beamWidth=10)

PDFPageCountError: ignored

In [ ]:
boxes = bounds[i][1]

In [ ]:
bounds[0]

In [ ]:
for box in boxes:
  cv2.rectangle(img, (int(box[0][0]), int(box[0][1])), (int(box[2][0]), int(box[2][1])), (0,0,255),1)
cv2.imwrite('detection_'+ str(i+1) + '.jpg', img)


In [ ]:
text = []
for i in range(len(bounds)):
  text.append(bounds[i][1]+'\n')

In [ ]:
pd.DataFrame(text).to_csv('output')

In [ ]:
def process_image(image_path):
  reader = easyocr.Reader(['en'])
  image = convert_from_path(image_path, dpi=300 )

  bounds = reader.readtext(np.array(image[0]), min_size=0, slope_ths=0.2, ycenter_ths=0.6, width_ths= 0.7, decoder='beamsearch', beamWidth=10)
  return (image[0], bounds)


In [ ]:
def draw_boxes(image, bounds, color='red', width=2):
  draw = ImageDraw.Draw(image)
  for i in bounds:
    p0, p1, p2, p3 = i[0]
    draw.line([*p0, *p1, *p2, *p3, *p0], fill=color, width=width)
    # draw.rectangle(((p3),(p0)), fill=color, width=width)
  return image


In [ ]:
def getText(bounds):
  text = ''
  for i in range(len(bounds)):
    text += bounds[i][1] +'\n'

  return text
#list = text.split('\n')
#print(list)


In [ ]:
def heading(text):

  company_name = re.findall(r'\bHENG YANG VALIN STEEL TUBE CO\b', text)
  address = re.findall(r'\bHengyang City,Hunan,PR China PC\b', text)
  email = re.findall(r'(\b\w+@\w+\.\w+\b)', text)


  info = [("Company Name", company_name),
            ("Address", address),
            ("Email", email)]

  for key, value in info:
    print(key ,':', value)

  return(info)

In [ ]:
key_list_awaji = ['Purchaser',
                'Date',
                'Certificate No',
                'Order No',
                'Product',
                'Pipe Heat No.',
                'Material',
                'Article & Size'
                ]

new_key_list_awaji = []
for key in key_list_awaji:
  key = key.replace('\n','')
  key = key.replace(' ','_')
  key = key.lower()
  key = key.strip()
  new_key_list_awaji.append(key)


def extract_data(text):

  purchaser_pattern = r'Purchaser\n(\w+ .*)'
  date_pattern = r'\d{2}\/\d{2}\/\d{4}'
  certificate_pattern = r'Certificate No\s(\w+-\d+)'
  product_pattern = re.compile(r'STAINLESS STEEL BUTT WELDING FITTINGS')
  material_pattern = r'(\w{3,}\/\w{3,})'
  pipe_heat_no_pattern = r'Pipe Heat No\.\n(\d+)'
  article_size_pattern = r'(\w+ \d{3}\/\d+.*)'

  purchaser = re.search(purchaser_pattern, text).group(1)
  date = re.search(date_pattern, text).group()
  certificate = re.search(certificate_pattern, text).group(1)
  order_number = re.search(r'\d{10}', text).group()
  product = re.search(product_pattern,text).group()
  material = re.search(material_pattern,text).group()
  pipe_heat_no = re.search(pipe_heat_no_pattern,text).group(1)
  article_size = re.search(article_size_pattern,text).group()


  extracted_data = [
      ("Purchaser", purchaser),
      ("Date", date),
      ("Certificate No", certificate),
      ("Order No", order_number),
      ("Product", product),
      ("Material", material),
      ("Pipe Heat No.", pipe_heat_no),
      ("Article & Size", article_size)
  ]

  return extracted_data


In [ ]:
key_list = ['CERTIFICATENQ',
            'Company Name',
            'CUSTOMER',
            'PONO',
            'DELIVERY CONDITION',
            'PRODUCT',
            'PROCESS OF MANUFACTURE',
            'HEAT TREATMENT',
            'SPECIFICATION',
            'PIPE-END DESCRIBE',
            'DIMENSIONS',
            'GRADE',
            'TOTAL BUNDLES',
            'TOTAL PIECES',
            'TOTAL LENGTH']

new_key_list = []
for key in key_list:
  key = key.replace('\n','')
  key = key.replace(' ','_')

  key = key.lower()
  key = key.strip()
  new_key_list.append(key)

In [ ]:
new_key_list

In [ ]:
def main(new_key_list):
  reader = easyocr.Reader(['en'])
  path = "/content/"
  file = "Hengyang-2.pdf"
  image_path = path+file
  image = convert_from_path(image_path, dpi=300 )

  #result = {}
  for i in image:
    bounds = reader.readtext(image, min_size=0, slope_ths=0.2, ycenter_ths=0.6, width_ths= 0.7, decoder='beamsearch', beamWidth=10)

    text = ''
    for i in range(len(bounds)):
      text += bounds[i][1]+'\n'

    #print(text)


    if(n==1):
      head_extract = heading(text)
    print('\n')


    key_value_pairs_hengyang = re.findall(r'([A-Z\s|-]+):\s+(.+)', text)
    #key_value_pairs_awaji = extract_data(text)
    #key_value_pairs = re.findall(r'(\w+):\s*([\w\s\d\'/]+)?', text)

    #print('\n', "page: ",n ,'\n')



    #print(result)
    for key, value in key_value_pairs_hengyang:
      key = key.replace('\n','')
      key = key.replace(' ','_')
      key = key.lower()
      key = key.strip()

      if key in new_key_list:
        print(key, ':', value,'\n')
    return 0



    #result_page = {key:value for key, value in key_value_pairs}
    #min_max = {}


      #min_max = getMinMax(text)
      #result.update(min_max)
    #result.update(result_page)
    #print(result_page)
    #n = n+1
  #print(result)
  #return result






In [ ]:
if __name__ == "__main__":
  main(new_key_list)

In [ ]:
import re

text = "TRAIN NO:\nDATE:\n20200210\nCONTRACT-ITEM NO.:\n2319110005012"

pattern = re.compile(r"TRAIN NO:\s+(\d+)")

match = re.search(pattern, text)
if match:
    train_no = match.group(1)
    print("Train No:", train_no)
else:
    print("Train No not found.")


In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr

In [ ]:
text = pytesseract.image_to_string(img[0])
print(text)